In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from sklearn import preprocessing 
from category_encoders import *
from sklearn.preprocessing import LabelEncoder
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error
from sklearn import datasets, linear_model, metrics
from sklearn.metrics import  confusion_matrix

# Loading Data

## Here first we will load train data

In [ ]:
df = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
df

# Exploratory Data Analysis

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
df.size

In [ ]:
df.dtypes

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [ ]:
df.skew()

In [ ]:
df.corr()

In [ ]:
df.isnull().sum()

In [ ]:
# Droping thesee columns since they have many null values
df.drop(['PoolQC','Fence','MiscFeature','FireplaceQu','Alley'],axis = 1,inplace = True)

In [ ]:
# Column with null values and their count
c = 0
is_null = []
for i in df.columns:
    if df[i].isnull().sum()>0:
        is_null.append(i)
        print(i,df[i].isnull().sum())
        c = c+1
print('Number of columns containing null values are:',c)

In [ ]:
# 38 categorical columns
obj = []
for i in df.columns:
    if df[i].dtypes == 'object':
        obj.append(i)
obj#categorical column list

# Data Visualisation

In [ ]:
# count of categories in categorical columns
for i in obj:
    print(df[i].value_counts())
    print('-----------------------------------------------------')

In [ ]:
# count plot of categorical values in each column
for i in obj:
    sns.countplot(x = i ,data=df)
    plt.show()

### Target variable vs all categorical columns

In [ ]:
for i in obj:
    data=df.copy()
    data.groupby(i)['SalePrice'].median().plot.bar()
    plt.xlabel(i)
    plt.ylabel('Sale Price')
    plt.title(i)
    plt.show()

### hist plot of all numerical columns

In [ ]:
plt.figure(figsize=(6,8))
x = df.drop(obj,axis = 1)
for i in x.columns:
    sns.histplot(x[i],kde = True)
    plt.show()

### Target variable vs numerical columns

In [ ]:
plt.figure(figsize=(6,8))
x = df.drop(obj,axis = 1)
for i in x.columns:
    sns.scatterplot(x = 'SalePrice',y = i,data = x,color = 'Red')
    plt.show()

### Heat map of corelation graph

In [ ]:
plt.figure(figsize=(25,16))
ax = sns.heatmap(df.corr(),annot = True, cmap = 'viridis')
plt.show()

### Box plot to observe outliers

In [ ]:
x = df.drop(obj,axis = 1)
for i in x.columns:
    sns.boxplot(x = i, data = x,color = 'yellowgreen')   
    plt.xlabel(i)
    plt.show()

### violin plot

In [ ]:
x = df.drop(obj,axis = 1)
for i in x.columns:
    sns.violinplot(x = i, data = x,color = 'yellowgreen')   
    plt.xlabel(i)
    plt.show()

# Count of outliers in each numerical columns

In [ ]:
def count_outliers(data,col):
        q1 = data[col].quantile(0.25,interpolation='nearest')
        q2 = data[col].quantile(0.5,interpolation='nearest')
        q3 = data[col].quantile(0.75,interpolation='nearest')
        q4 = data[col].quantile(1,interpolation='nearest')
        IQR = q3 -q1
        global LLP
        global ULP
        LLP = q1 - 1.5*IQR
        ULP = q3 + 1.5*IQR
        if data[col].min() > LLP and data[col].max() < ULP:
            print("No outliers in",i)
        else:
            print("There are outliers in",i)
            x = data[data[col]<LLP][col].size
            y = data[data[col]>ULP][col].size
            a.append(i)
            print('Count of outliers are:',x+y)
global a
a = []
for i in x.columns:
    count_outliers(df,i)

# Data Preprocessing

### Filling Null Values

### Filling null values with median if there are outliers in that particular column
### Filling null values with mean if there are no outliers in that particular column
### Filling null vaues with mode if that column is categorical column

In [ ]:
for i in is_null:
    if df[i].dtype == 'int64' or df[i].dtype == 'float64':
        if i in a:
                df[i].fillna(df[i].median(),inplace=True)
        else:
            df[i].fillna(df[i].mean(),inplace=True)
    elif df[i].dtype == 'object':
        df[i].fillna(df[i].mode()[0],inplace=True)

In [ ]:
# No null values
df.info()

### Encoding

In [ ]:
df1=pd.get_dummies(data=df,columns=obj,drop_first=True)
df1

# Cleaning and preprocessing of Test dataset

## Before applying model clean and setup the test dataset same as train for prediction

In [ ]:
df2 = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')
df2

In [ ]:
#Droping columns with more null values
df2.drop(['PoolQC','Fence','MiscFeature','FireplaceQu','Alley'],axis = 1,inplace = True)
df2

In [ ]:
c1 = 0
is_null1 = []
for i in df2.columns:
    if df2[i].isnull().sum()>0:
        is_null1.append(i)
        print(i,df2[i].isnull().sum())
        c1 = c1+1
print('Number of columns containing null values are:',c1)

In [ ]:
# 38 categorical columns
obj1 = []
for i in df2.columns:
    if df2[i].dtypes == 'object':
        obj1.append(i)
len(obj1)

In [ ]:
x1 = df2.drop(obj,axis = 1)
def count_outliers(data,col):
        q1 = data[col].quantile(0.25,interpolation='nearest')
        q2 = data[col].quantile(0.5,interpolation='nearest')
        q3 = data[col].quantile(0.75,interpolation='nearest')
        q4 = data[col].quantile(1,interpolation='nearest')
        IQR = q3 -q1
        global LLP
        global ULP
        LLP = q1 - 1.5*IQR
        ULP = q3 + 1.5*IQR
        if data[col].min() > LLP and data[col].max() < ULP:
            print("No outliers in",i)
        else:
            print("There are outliers in",i)
            x = data[data[col]<LLP][col].size
            y = data[data[col]>ULP][col].size
            k.append(i)
            print('Count of outliers are:',x+y)
global k
k = []
for i in x1.columns:
    count_outliers(df2,i)

In [ ]:
for i in is_null1:
    if df2[i].dtype == 'int64' or df2[i].dtype == 'float64':
        if i in k:
                df2[i].fillna(df2[i].median(),inplace=True)
        else:
            df2[i].fillna(df2[i].mean(),inplace=True)
    elif df2[i].dtype == 'object':
        df2[i].fillna(df2[i].mode()[0],inplace=True)

In [ ]:
df2.info()

In [ ]:
df3=pd.get_dummies(data=df2,columns=obj1,drop_first=True)
df3

### This is a list which has column names that doesnot present in test dataframe

In [ ]:
v = []
for i in df1.columns:
    if i not in df3.columns:
        v.append(i)
v

# Feature Selection

In [ ]:
X = df1.drop(v,axis = 1)#dropping columns that doesnot present in test dataset
Y = df1['SalePrice']
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3,random_state=44)

# Linear Regression prediction

In [ ]:
reg = linear_model.LinearRegression()
reg.fit(X_train, Y_train)

In [ ]:
reg.coef_

In [ ]:
pred = reg.predict(X_test)
pred

In [ ]:
plt.scatter(Y_test,pred)
plt.xlabel('Y Test (True Values)')
plt.ylabel('Predicted values')
plt.show()

In [ ]:
#r2 score
metrics.explained_variance_score(Y_test,pred)

In [ ]:
# This shows data is normally distributed but right skewd
sns.displot(Y_test-pred,bins = 50,kde = True)
plt.show()

### This states that 1 unit increase in saleprice will decrease LotFrontage by 102.622826 units

In [ ]:
cdf = pd.DataFrame(reg.coef_,X.columns,columns = ['coef'])
cdf

# Regression prediction for test dataset

In [ ]:
pred1 = reg.predict(df3)
pred1

In [ ]:
# List contains values of Id columns
f = []
for i in df3.Id:
    f.append(i)


### Displaying regression predictions of test dataset in dataframe

In [ ]:
p = pd.DataFrame(data = [pred1])
d = p.transpose()
d.insert(loc = 0, column = 'Id', value=f, allow_duplicates = False)
d.rename(columns={0: 'SalePrice'},inplace = True)
d.set_index('Id')


# Random forest prediction of train dataset

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest= RandomForestRegressor(n_estimators =40, random_state = 0)
forest.fit(X_train,Y_train)  
Y_pred = forest.predict(X_test)
forest.score(X_test,Y_test)

# Random forest prediction of test dataset

In [ ]:
pred2 = forest.predict(df3)
pred2

In [ ]:
p = pd.DataFrame(data = [pred2])
d = p.transpose()
d.insert(loc = 0, column = 'Id', value=f, allow_duplicates = False)

## Displaying random forest predictions of test dataset in dataframe

In [ ]:
d.rename(columns={0: 'SalePrice'},inplace = True)
d.set_index('Id')

# Xg boost prediction on train data

In [ ]:
import xgboost as xg
xg_reg = xg.XGBRegressor(objective ='reg:squarederror',
                  n_estimators = 50, seed = 123)
xg_reg.fit(X_train, Y_train)
pred = xg_reg.predict(X_test)
print(xg_reg.score(X_test, Y_test))

# Xg boost prediction on test data

## Displaying random forest predictions of test dataset in dataframe

In [ ]:
p = pd.DataFrame(data = [pred2])
d = p.transpose()
d.insert(loc = 0, column = 'Id', value=f, allow_duplicates = False)
d.rename(columns={0: 'SalePrice'},inplace = True)
d.set_index('Id')